In [ ]:
from WikiArticleXmlHandler import WikiArticleXmlHandler
from WikiDataXmlHandler import WikiDataXmlHandler
import xml.sax
import mwparserfromhell
import stanfordnlp

import subprocess
import pandas as pd
import os
import Utils

# from ProcessArticle import ProcessArticle

# WIKIDATA

In [ ]:
file = "./wikidata/wikidatawiki-latest-pages-articles-multistream1.xml-p1p235321.bz2"

In [ ]:
%%time

handler = WikiDataXmlHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}\n".format(file))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(file),
                        stdout=subprocess.PIPE).stdout:
    
    if handler._counter > 5: break
    parser.feed(line)

In [ ]:
for i in range(6):
    print(handler._pages[i][0])

In [ ]:
import json

a = json.loads(handler._pages[3][1])

In [ ]:
a.keys()

In [ ]:
a["labels"]["en"]["value"]

In [ ]:
# get wikidata instanceof
a["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["id"]

In [ ]:
for i in range(len(a["claims"]["P31"])):
    print(a["claims"]["P31"][i]["mainsnak"]["datavalue"]["value"]["id"])

In [ ]:
x = "|".join([a["claims"]["P31"][i]["mainsnak"]["datavalue"]["value"]["id"] for i in range(len(a["claims"]["P31"]))])
x

# ARTICLES

In [ ]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
out_file = "./saved/debug.csv"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp=nlp)

In [ ]:
%%time
file = "./test_data/test.xml"

handler = WikiArticleXmlHandler(out_file, func, 1000)
handler._DEBUG = True

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for line in open(file, "r", encoding="utf8"):
    parser.feed(line)

# handler.write_df()

In [ ]:
n_df = pd.read_csv(out_file, delimiter="\t")

In [ ]:
n_df

In [ ]:
n_df[n_df.title.str.contains("(?i)cobol")]

In [ ]:
for i, a in enumerate(handler._data):
    if a[0] == "List of artificial intelligence projects":
        print("{}: {}".format(i, a[0]))

In [ ]:
for i, a in enumerate(handler._data):
    # if a[0] == "COBOL":
    print("{}: {}".format(i, a[0]))

In [ ]:
page = handler._data[2]
text = handler._text[2]

In [ ]:
page[4].split("|")

In [ ]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
cols = ["title", "type", "def words", "infobox_name", "infobox", "categories","wikilinks"]

In [ ]:
wiki = mwparserfromhell.parse(text)

In [ ]:
a = wiki.strip_code().split("\n")
a = [" ".join(Utils.cleanhtml(x.strip()).split()) for x in a if not x.isspace() and x != ""]
a = a[0] # 1st paragraph
a

In [ ]:
nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

In [ ]:
doc = nlp(a)

In [ ]:
sentence = doc.sentences[0]

In [ ]:
sentence.words[14]

In [ ]:
import datetime

months_choices = list()
for i in range(1,13):
    months_choices.append((i, datetime.date(2008, i, 1).strftime('%B')))

months_choices

In [ ]:
import datetime

definitions = []
# no_no = ["punct", "nummod", "appos", "det", "nmod:poss", "cop", "cc", "nsubj", "case"]
no_no = ["punct", "nummod", "det"]
months = [datetime.date(2008, i, 1).strftime("%B").lower() for i in range(1,13)]

In [ ]:
print("index\tlemma\t\t\tgovernor\t\t\ttype\n")
for word in sentence.words:
    index = word.index.rjust(2)
    word_lemma = word.lemma
    governor = sentence.words[word.governor-1].lemma if word.governor > 0 else "root"
    deprel = word.dependency_relation

    if deprel in no_no or word_lemma.lower() in months:
        continue

    print("{}\t{}\t\t\t{}\t\t\t{}".format(index, word_lemma, governor, deprel))

In [ ]:
be_index = None
be_gov = None
a = []

for word in sentence.words:
    if word.lemma == "be":
        be_index = int(word.index.rjust(2))
        be_gov = word.governor
        a.append(sentence.words[be_gov-1].lemma)
        break

for word in sentence.words:
    if word.governor == be_gov and word.dependency_relation in ["amod", "conj", "compound"]:
        a.append(word.lemma)

In [ ]:
a

In [ ]:
print("{} {} {}".format(be_index, be_gov, be_deprel))

In [ ]:
# POS tags
print(" ". join(["{} ({})".format(word.text, word.upos) for word in sentence.words]))

In [ ]:
for word in sentence.tokens[14].words:
    if word.lemma in ["be"]:
        print("a")

In [ ]:
sentence.tokens[14].text

In [ ]:
print(*[f"index: {word.index.rjust(2)}\tword: {word.text.ljust(11)}\tgovernor index: {word.governor}\tgovernor: {(sentence.words[word.governor-1].text if word.governor > 0 else 'root').ljust(11)}\tdeprel: {word.dependency_relation}" for word in sentence.words], sep='\n')

In [ ]:
print(" ". join(["{} ({} - {})".format(word.text, word.upos, word.feats) for word in sentence.words]))

In [ ]:
sentence.build_dependencies()

In [ ]:
sentence.dependencies[0]

In [ ]:
wiki.filter_templates(matches="softredirect")

In [ ]:
"list of" in wiki.filter_templates(matches="DEFAULTSORT")[0].name.lower()

In [ ]:
# Creating dict
d = {col: [] for col in cols}
df = pd.DataFrame(columns=cols)
# parse page
# wiki = mwparserfromhell.parse(page[1])

In [ ]:
t = [None, "3", "None", "6", None, None,"1"]

In [ ]:
[d[cols[i]].append(t[i]) for i in range(len(t))]

In [ ]:
d

In [ ]:
df

In [ ]:
d

In [ ]:
# page title
# d["title"].append(page[0])
df["title"] = [page[0]]

In [ ]:
# defining article type
if page[2] != None:
    # d["type"].append("redirect")
    df["type"] = ["redirect"]
else:
    # d["type"].append("article")
    df["type"] = ["article"]

In [ ]:
# Getting article categories
# d["categories"].append(
#     "|".join([
#         list(filter(None, x.title.split(":")))[-1]
#         for x in wiki.filter_wikilinks(matches="category")
#     ]))
df["categories"] = ["|".join([
        list(filter(None, x.title.split(":")))[-1]
        for x in wiki.filter_wikilinks(matches="category")
    ])]

In [ ]:
# wikilinks, removing links to Category items
# d["wikilinks"].append("|".join(
#     [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()]))

df["wikilinks"] = ["|".join(
    [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()])]

# external links
# df["ext_links"] = "|".join([link.url.strip_code().strip() for link in wiki.filter_external_links()])

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
# search for infobox template name
box = None
for template in wiki.filter_templates():
    if template.name.strip() in infoboxes:
        box = wiki.filter_templates(matches=template.name.strip())[0]
        # d["infobox_name"].append(template.name.strip())
        df["infobox_name"] = [template.name.strip()]
        break

a = "|".join(["{}:{}".format(param.name.strip_code().strip(),
                            param.value.strip_code(keep_template_params=True).strip()) for param in box.params])
# d["infobox"].append(cleanhtml(a))
df["infobox"] = [cleanhtml(a)]

In [ ]:
df

In [ ]:
df["infobox"][0].split("|")

In [ ]:
df["wikilinks"][0].split("|")

In [ ]:
# mais proximo da solucao definitiva
for param in box.params:
    for x in param.value.filter_templates():
        print([y.value.strip_code() for y in x.params])

In [ ]:
# box.params.strip_code()
for param in box.params:
    # print(param.value.filter_text())
    # print(param.value.filter_wikilinks())
    print(param.value.strip_code(keep_template_params=True).strip())

In [ ]:
df["infobox"][0]